In [1]:
import datetime
import requests
import hyp3_sdk
import numpy as np
from tqdm import tqdm 
from copy import deepcopy

In [2]:
%load_ext autoreload
%autoreload 2

In [176]:
def isce_jobs4resubmit(jobs_dicts:list):
    job_definition = {'name': None,
                      'job_parameters': None,
                      'job_type': 'INSAR_ISCE'
                     }

    prepared_jobs = []
    for job in jobs_dicts:
        prepared_job = deepcopy(job_definition)
        prepared_job['name'] = job['name']
        prepared_job['job_parameters'] = job['job_parameters']
        prepared_job['job_type'] = job['job_type']
        prepared_jobs.append(prepared_job)
    return prepared_jobs

def raider_jobs4resubmit(jobs_dicts:list):
    job_definition = {
        'name': 're-run RAiDER step',  # optional: provide a name
        "job_type": "ARIA_RAIDER",
        "job_parameters": {
            # "job_id": "27836b79-e5b2-4d8f-932f-659724ea02c3",
            # "weather_model": "ERA5"
        },
    }

    prepared_jobs = []
    for job in jobs_dicts:
        prepared_job = deepcopy(job_definition)
        prepared_job['name'] = job['name'] + '_rerun_raider'
        prepared_job['job_parameters']['job_id'] = job['job_id']
        prepared_job['job_parameters']['weather_model'] = job['job_parameters']['weather_model'] 
        prepared_jobs.append(prepared_job)
    return prepared_jobs

def split_failed_jobs(failed_jobs):
    flag_insar_job = lambda x, y: (x.job_type == 'INSAR_ISCE') & (len(x.processing_times)==y)
    # isce
    isce_jobs = [j.to_dict() for j in failed_jobs if flag_insar_job(j, 1)]
    # raider
    raider_jobs_new = [j.to_dict() for j in failed_jobs if flag_insar_job(j, 2)] 
    raider_jobs = [j.to_dict() for j in failed_jobs if j.job_type == 'ARIA_RAIDER']
    raider_jobs = raider_jobs_new + raider_jobs
    return isce_jobs, raider_jobs

def get_request_date(job_dict):
    job_date = datetime.datetime.strptime(job_dict['request_time'], '%Y-%m-%dT%H:%M:%S%z')
    return datetime.datetime.strftime(job_date,'%Y-%d-%m')  

# TODO: add function to find duplicate jobs to filter out same jobs
# for now track it by date
class access_hyp3_resubmit():
    def __init__(self, hyp3_jobs):
        # Filter to failed jobs
        failed_jobs = hyp3_jobs.filter_jobs(succeeded=False, pending=False,
                                            running=False, failed=True)
        print(f'All jobs:\n {failed_jobs}')

        # Failed at topssApp step
        self.isce_jobs, self.raider_jobs = split_failed_jobs(failed_jobs)
        print(f'Topsapp jobs: {len(self.isce_jobs)}')
        print(f' Sumbission dates: {np.unique(list(map(get_request_date, self.isce_jobs)))}')

        # Failed at raider step
        print(f'Raider jobs: {len(self.raider_jobs)}')
        print(f' Sumbission dates: {np.unique(list(map(get_request_date, self.raider_jobs)))}')

    def prepare_jobs(self, batch_size=200):
        self.prepared_isce = isce_jobs4resubmit(self.isce_jobs)
        self.prepared_raider = raider_jobs4resubmit(self.raider_jobs)
        # isce
        self.isce_resubmit = [batch for batch in hyp3_sdk.util.chunk(self.prepared_isce, batch_size)]
        msg = f'Prepared isce jobs: {len(self.prepared_isce)} in {len(self.isce_resubmit[0])}'
        msg += f'x {len(self.isce_resubmit)} batches'
        print(msg) 
        # raider
        self.raider_resubmit = [batch for batch in hyp3_sdk.util.chunk(self.prepared_raider, batch_size)]
        msg = f'Prepared raider jobs: {len(self.prepared_raider)} in {len(self.raider_resubmit[0])}'
        msg += f'x {len(self.raider_resubmit)} batches'
        print(msg)
        
    def submit_isce(self):
        print(f'Submitting: {len(self.prepared_isce)} jobs')
        for batch in tqdm(self.isce_resubmit):
            hyp3_isce.submit_prepared_jobs(batch)

    def submit_raider(self):
        print(f'Submitting: {len(self.prepared_raider)} jobs')
        for batch in tqdm(self.raider_resubmit):
            hyp3_isce.submit_prepared_jobs(batch) 


## Setup

In [4]:
# uses .netrc; add `prompt=True` to prompt for credentials;
hyp3_isce = hyp3_sdk.HyP3('https://hyp3-a19-jpl.asf.alaska.edu', prompt=True)
# hyp3_isce = hyp3_sdk.HyP3('https://hyp3-tibet-jpl.asf.alaska.edu', prompt=True)

In [142]:
# Since the campaign first date
jobs = hyp3_isce.find_jobs(user_id='access_cloud_based_insar',
                           start=datetime.datetime(2024, 2, 6), 
                           end=datetime.datetime(2024, 5, 21))
print(jobs)

219029 HyP3 Jobs: 92785 succeeded, 124106 failed, 1001 running, 1137 pending.


In [144]:
# Since I started
jobs = hyp3_isce.find_jobs(start=datetime.datetime(2024, 4, 25), 
                           end=datetime.datetime(2024, 5, 21),
                           user_id='access_cloud_based_insar')
print(jobs)

98418 HyP3 Jobs: 38804 succeeded, 57476 failed, 1003 running, 1135 pending.


Track jobs by the submission dates

In [150]:
jobs = hyp3_isce.find_jobs(start=datetime.datetime(2024, 5, 5), 
                           end=datetime.datetime(2024, 5, 21),
                           user_id='access_cloud_based_insar')
print(jobs)

79343 HyP3 Jobs: 26274 succeeded, 50931 failed, 1003 running, 1135 pending.


In [151]:
failed_jobs_flag = dict(succeeded=False, pending=False,
                        running=False, failed=True)
failed_jobs = jobs.filter_jobs(**failed_jobs_flag)

In [178]:
i, r = split_failed_jobs(failed_jobs)

In [182]:
jobs_id = [x['job_id'] for x in r]

In [170]:
hash(tuple(failed_jobs[2].job_parameters.get('granules')))

-4815360217136865304

In [119]:
resumbission_jobs = access_hyp3_resubmit(failed_jobs)
resumbission_jobs.prepare_jobs()

All jobs:
 50868 HyP3 Jobs: 0 succeeded, 50868 failed, 0 running, 0 pending.
Topsapp jobs: 2201
 Sumbission dates: ['2024-05-05']


Raider jobs: 48667
 Sumbission dates: ['2024-05-05' '2024-07-05']
Prepared isce jobs: 2201 in 200x 12 batches
Prepared raider jobs: 48667 in 200x 244 batches


In [121]:
resumbission_jobs.submit_isce()

Submitting: 2201 jobs


  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:23<00:00,  1.95s/it]


In [127]:
jobs = hyp3_isce.find_jobs(user_id='access_cloud_based_insar',
                           start=datetime.datetime(2024, 5, 6), 
                           end=datetime.datetime(2024, 5, 21))
failed_jobs_flag = dict(succeeded=False, pending=False,
                        running=False, failed=True)

print('#' * 10)
print(jobs)
print('#' * 10)
rjobs = access_hyp3_resubmit(jobs.filter_jobs(**failed_jobs_flag))

##########
26532 HyP3 Jobs: 3 succeeded, 24330 failed, 234 running, 1965 pending.
##########
All jobs:
 24330 HyP3 Jobs: 0 succeeded, 24330 failed, 0 running, 0 pending.
Topsapp jobs: 2
 Sumbission dates: ['2024-07-05']
Raider jobs: 24328
 Sumbission dates: ['2024-07-05']


In [133]:
## Check failed jobs
all_logs = [j['logs'] for j in rjobs.raider_jobs]
all_logs[0], all_logs[-1]

(['https://hyp3-a19-jpl-contentbucket-1wfnatpznlg8b.s3.us-west-2.amazonaws.com/6716cbda-0088-4262-920a-5f5287a4dbc7/6716cbda-0088-4262-920a-5f5287a4dbc7.log'],
 ['https://hyp3-a19-jpl-contentbucket-1wfnatpznlg8b.s3.us-west-2.amazonaws.com/9d27256a-acf3-4ed9-92b0-c0d04374a6c8/9d27256a-acf3-4ed9-92b0-c0d04374a6c8.log'])

In [135]:
# Get log last 3 lines
'''
failed_logs = []
for ix, log in enumerate(all_logs):
    r = requests.get(log)
    failed_logs.append({ix: r.text.splitlines()[-3:]})'''

'\nfailed_logs = []\nfor ix, log in enumerate(all_logs):\n    r = requests.get(log)\n    failed_logs.append({ix: r.text.splitlines()[-3:]})'

## asf search

In [136]:
import asf_search as asf
import geopandas as gpd

In [140]:
# requires asf_search > 7.0.8
opts = asf.ASFSearchOptions(
    shortName='ARIA_S1_GUNW',
    #relativeOrbit=[13],
    #flightDirection='DESCENDING',
    #processingLevel='GUNW_STD',
    intersectsWith='POLYGON((-127.992 32.2865,-115.5115 32.2865,-115.5115 49.1812,-127.992 49.1812,-127.992 32.2865))',
    collectionAlias=False,
)

In [141]:
scenes = asf.search(opts=opts)
gdf = gpd.GeoDataFrame.from_features(scenes.geojson(), crs='EPSG:4326')

In [147]:
gdf.shape

(44594, 30)

In [143]:
version = gdf.fileID.apply(lambda x: int(x.split('-')[-1].split('_')[0].split('v')[-1]))
version.unique()

array([3])